In [1]:
old_druggable_file_path = "../DRUG_INFO/druggable_proteins.txt"
old_investigational_file_path = "../DRUG_INFO/investigational_proteins.txt"
new_druggable_file_path = "../NEW_DRUG_INFO/druggable_proteins.txt"
new_investigational_file_path = "../NEW_DRUG_INFO/investigational_proteins.txt"

def get_protein_list(file_path):
    with open(file_path, "r") as f:
        return [line.strip() for line in f]

old_druggable_proteins = get_protein_list(old_druggable_file_path)
old_investigational_proteins = get_protein_list(old_investigational_file_path)
new_druggable_proteins = get_protein_list(new_druggable_file_path)
new_investigational_proteins = get_protein_list(new_investigational_file_path)
len(old_druggable_proteins), len(old_investigational_proteins), len(new_druggable_proteins), len(new_investigational_proteins)

(2233, 682, 2310, 766)

In [2]:
# all old druggable are in new druggable
for protein in old_druggable_proteins:
    if protein not in new_druggable_proteins:
        print(protein)

O96007
P05120
P09681
Q01362


In [3]:
# newly druggable
newly_druggable_proteins = [protein for protein in new_druggable_proteins if protein not in old_druggable_proteins]

# of the newly druggable, how many are in the old investigational
newly_druggable_in_old_investigational = [protein for protein in newly_druggable_proteins if protein in old_investigational_proteins]
len(newly_druggable_in_old_investigational), len(newly_druggable_proteins)

(31, 81)

In [4]:
newly_investigational_proteins = [protein for protein in new_investigational_proteins if protein not in old_investigational_proteins]
len(newly_investigational_proteins)

115

In [10]:
import pandas as pd 
rf_di_investigational = pd.read_csv("RF_DI_investigational.csv", index_col=0)[['Mean_Probability', 'Majority_Prediction']]
rf_di_non_druggable = pd.read_csv("RF_DI_non_druggable.csv", index_col=0)[['Mean_Probability', 'Majority_Prediction']]
xgb_di_investigational = pd.read_csv("XGB_DI_investigational.csv", index_col=0)[['Mean_Probability', 'Majority_Prediction']]
xgb_di_non_druggable = pd.read_csv("XGB_DI_non_druggable.csv", index_col=0)[['Mean_Probability', 'Majority_Prediction']]

In [17]:
newly_druggable_scores = {}
for protein in newly_druggable_proteins:
    if protein in newly_druggable_in_old_investigational:
        assert protein in rf_di_investigational.index
        assert protein in xgb_di_investigational.index
        newly_druggable_scores[protein] = {
            'RF_Mean_Probability': rf_di_investigational.loc[protein, 'Mean_Probability'],
            'RF_Majority_Prediction': rf_di_investigational.loc[protein, 'Majority_Prediction'],
            'XGB_Mean_Probability': xgb_di_investigational.loc[protein, 'Mean_Probability'],
            'XGB_Majority_Prediction': xgb_di_investigational.loc[protein, 'Majority_Prediction'],
            'earlier': 'investigational'
        }
    else:
        assert protein in rf_di_non_druggable.index
        assert protein in xgb_di_non_druggable.index
        newly_druggable_scores[protein] = {
            'RF_Mean_Probability': rf_di_non_druggable.loc[protein, 'Mean_Probability'],
            'RF_Majority_Prediction': rf_di_non_druggable.loc[protein, 'Majority_Prediction'],
            'XGB_Mean_Probability': xgb_di_non_druggable.loc[protein, 'Mean_Probability'],
            'XGB_Majority_Prediction': xgb_di_non_druggable.loc[protein, 'Majority_Prediction'],
            'earlier': 'non_druggable'
        }


In [18]:
newly_druggable_scores = pd.DataFrame(newly_druggable_scores).T
newly_druggable_scores.to_csv("newly_druggable_scores.csv")

In [20]:
# sanity checks on newly_investigational_proteins
for protein in newly_investigational_proteins:
    if protein in old_druggable_proteins:
        print(protein)

P05120


In [21]:
# sanity checks on old_investigational_proteins
for protein in old_investigational_proteins:
    if protein in new_druggable_proteins or protein in new_investigational_proteins:
        continue
    else:
        print(protein)

In [22]:
newly_investigational_scores = {}
for protein in newly_investigational_proteins:
    if protein == "P05120": continue
    assert protein in rf_di_non_druggable.index
    assert protein in xgb_di_non_druggable.index
    newly_investigational_scores[protein] = {
        'RF_Mean_Probability': rf_di_non_druggable.loc[protein, 'Mean_Probability'],
        'RF_Majority_Prediction': rf_di_non_druggable.loc[protein, 'Majority_Prediction'],
        'XGB_Mean_Probability': xgb_di_non_druggable.loc[protein, 'Mean_Probability'],
        'XGB_Majority_Prediction': xgb_di_non_druggable.loc[protein, 'Majority_Prediction'],
    }

newly_investigational_scores = pd.DataFrame(newly_investigational_scores).T
newly_investigational_scores.to_csv("newly_investigational_scores.csv")